# RD performance of some color transforms

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command

import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
!ln -sf ~/MRVC/src/deadzone.py .
import deadzone as Q
!ln -sf ~/MRVC/src/frame.py .
import frame
!ln -sf ~/MRVC/src/YCrCb.py .
import YCrCb
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg

In [ ]:
def q_deq(x, quantization_step):
    k = Q.quantize(x, quantization_step)
    y = Q.dequantize(k, quantization_step)
    return k, y

# Notice that the dead-zone to have effect,
# the samples should be allowed to negative.

In [ ]:
def load_indexes(prefix):
    frame.read(prefix)
    
def write_indexes(prefix):
    frame.write(prefix)

In [ ]:
fn = "/home/vruiz/MRVC/sequences/stockholm_5_frames/000"
img = frame.read(fn)

In [ ]:
def show(img, title):
    img = frame.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img)

In [ ]:
show(img, "stockholm000")

In [ ]:
def show_gray(img, title):
    img = frame.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img, cmap='gray')

In [ ]:
show_gray(img[:,:,0], "stockholm (R component)")

In [ ]:
show_gray(img[:,:,1], "stockholm0000 (G component)")

In [ ]:
show_gray(img[:,:,2], "stockholm000 (B component)")

In [ ]:
def RGB_to_YCrCb(RGB_frame):
    # Remember that cv2.cvtColor only works with unsigneds!
    YCrCb_frame = cv2.cvtColor(RGB_frame, cv2.COLOR_RGB2YCR_CB)
    return YCrCb_frame

def YCrCb_to_RGB(YCrCb_frame):
    RGB_frame = cv2.cvtColor(YCrCb_frame, cv2.COLOR_YCR_CB2RGB)
    return RGB_frame

def RGB_to_YCoCg(RGB_frame):
    R, G, B = RGB_frame[:,:,0], RGB_frame[:,:,1], RGB_frame[:,:,2]
    YCoCg_frame = np.empty_like(RGB_frame)
    YCoCg_frame[:,:,0] =  R/4 + G/2 + B/4 
    YCoCg_frame[:,:,1] =  R/2       - B/2
    YCoCg_frame[:,:,2] = -R/4 + G/2 - B/4
    return YCoCg_frame

def YCoCg_to_RGB(YCoCg_frame):
    Y, Co, Cg = YCoCg_frame[:,:,0], YCoCg_frame[:,:,1], YCoCg_frame[:,:,2]
    RGB_frame = np.empty_like(YCoCg_frame)
    RGB_frame[:,:,0] = Y + Co - Cg 
    RGB_frame[:,:,1] = Y      + Cg
    RGB_frame[:,:,2] = Y - Co - Cg
    return RGB_frame

### Is the YCrCb reversible?

In [ ]:
YCrCb_img = YCrCb.from_RGB(img.astype(np.uint8))
print(YCrCb_img.shape)
img2 = YCrCb.to_RGB(YCrCb_img)
print(np.array_equal(img, img2))
print(img.max(), img.min())
print(YCrCb_img.max(), YCrCb_img.min(), np.average(YCrCb_img))

In [ ]:
show_gray(YCrCb_img[:,:,0], "stockholm000 (Y component, YCrCb domain)")

In [ ]:
show_gray(YCrCb_img[:,:,1], "stockholm000 (Cr component, YCrCb domain)")

In [ ]:
show_gray(YCrCb_img[:,:,2], "stockholm000 (Cb component)")

### Is the YCoCg reversible?

In [ ]:
YCoCg_img = YCoCg.from_RGB(img.astype(np.int16))
img2 = YCoCg.to_RGB(YCoCg_img)
print(np.array_equal(img, img2))
print(img.max(), img.min())
print(YCoCg_img.max(), YCoCg_img.min(), np.average(YCoCg_img))

In [ ]:
show(img, "stockholm000 (original)")

In [ ]:
show(img2, "stockholm000 (reconstructed through YCoCg)")

In [ ]:
show(img-img2, 'stockholm000 - YCoCg.to\_RGB(YCoCg.from\_RGB(stockholm000))')

In [ ]:
show_gray(YCoCg_img[:,:,0], "stockholm000 (Y component)")

In [ ]:
show_gray(YCoCg_img[:,:,1], "stockholm000 (Co component)")

In [ ]:
show_gray(YCoCg_img[:,:,2], "stockholm000 (Cg component)")

## RD stuff

### Rate measurement

In [ ]:
def bytes_per_frame(img):
    frame.write(img, "/tmp/frame")
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def bytes_per_grayframe(img):
    cv2.imwrite("/tmp/frame.png", img)
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

### Distortion measurement

In [ ]:
def average_energy(x):
    return np.sum(x.astype(np.double)*x.astype(np.double))/(np.size(x))

def MSE(x, y):
    error_signal = x.astype(np.float32) - y
    return average_energy(error_signal)

def RMSE(x, y):
    error_signal = x.astype(np.float32) - y
    return math.sqrt(MSE(error_signal))

## Gains of the YCrCb synthesis filters 

In [ ]:
val = 10
delta_YCrCb = np.array([val, 0, 0]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Y gain =", average_energy(delta_RGB))

delta_YCrCb = np.array([0, val, 0]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Cr gain =", average_energy(delta_RGB))

delta_YCrCb = np.array([0, 0, val]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Cb gain =", average_energy(delta_RGB))

delta_YCrCb = np.array([val, val, val]).astype(np.uint8).reshape(1,1,3)
delta_RGB = YCrCb.to_RGB(delta_YCrCb)
print("delta_YCrCb =", delta_YCrCb, "delta_RGB =", delta_RGB, "Total gain =", average_energy(delta_RGB))

As we can see, the energy of the components in the YCrCb domain is not additive (the same happens with the distortion generated by the quantization).

In [ ]:
val = 10
delta_YCoCg = np.array([val, 0, 0]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Y gain =", average_energy(delta_RGB))

delta_YCoCg = np.array([0, val, 0]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Cr gain =", average_energy(delta_RGB))

delta_YCoCg = np.array([0, 0, val]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Cb gain =", average_energy(delta_RGB))

delta_YCoCg = np.array([val, val, val]).reshape(1,1,3)
delta_RGB = YCoCg.to_RGB(delta_YCoCg)
print("delta_YCoCg =", delta_YCoCg, "delta_RGB =", delta_RGB, "Total gain =", average_energy(delta_RGB))

With the YCoCg happens the same (the energy of the components is not additive). The gains matches with the theory.

### Energy of the RGB channels

In [ ]:
R_energy = average_energy(img[:,:,0])
G_energy = average_energy(img[:,:,1])
B_energy = average_energy(img[:,:,2])
print("Energy of R =", R_energy)
print("Energy of G =", G_energy)
print("Energy of B =", B_energy)

### Energy of the YCrCb channels

In [ ]:
YCrCb_frame = YCrCb.from_RGB(img.astype(np.uint8))
Y_energy = average_energy(YCrCb_frame[:,:,0])
Cr_energy = average_energy(YCrCb_frame[:,:,1])
Cb_energy = average_energy(YCrCb_frame[:,:,2])
print("Energy of Y =", Y_energy)
print("Energy of Cr =", Cr_energy)
print("Energy of Cb =", Cb_energy)

### Energy of the YCoCg channels

In [ ]:
YCoCg_frame = YCoCg.from_RGB(img.astype(np.int16))
Y_energy = average_energy(YCoCg_frame[:,:,0])
Co_energy = average_energy(YCoCg_frame[:,:,1])
Cg_energy = average_energy(YCoCg_frame[:,:,2])
print("Energy of Y =", Y_energy)
print("Energy of Co =", Co_energy)
print("Energy of Cg =", Cg_energy)

The energy is more concentrated in the YCoCg domain, more specifically in the Y channel.

### Quantization in the RGB domain ($\Delta_{\text{R}} = \Delta_{\text{G}} = \Delta_{\text{B}}$)

In [ ]:
def RGB_RD_curve(x):
    points = []
    for q_step in range(0, 8):
        k, y = q_deq(x, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_frame(k)
        distortion = MSE(x, y)
        points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return points

RGB_points = RGB_RD_curve(img)

### RD using quantization in the YCbCr domain ($\Delta_{\text{Y}} = \Delta_{\text{Cr}} = \Delta_{\text{Cb}}$)

In [ ]:
def YCrCb_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        YCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        #YCrCb_frame = YCrCb_frame.astype(np.int16)
        k, dequantized_YCrCb_frame = q_deq(YCrCb_frame, 1<<q_step)
        k = k.astype(np.uint8)
        #show(dequantized_YCrCb_frame, q_step)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YCrCb_frame.astype(np.uint8))
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

YCrCb_points = YCrCb_RD_curve(img)

### RD curves of each YCrCb channel

In [ ]:
def only_Y_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        Y_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))[:,:,0]
        dequantized_Y_frame = np.empty_like(Y_frame)
        k = np.empty_like(Y_frame, dtype=np.uint8)
        k, dequantized_Y_frame = q_deq(Y_frame, 1<<q_step)
        rate = bytes_per_grayframe(k)
        distortion = MSE(Y_frame, dequantized_Y_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_Cr_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        Cr_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))[:,:,1]
        dequantized_Cr_frame = np.empty_like(Cr_frame)
        k = np.empty_like(Cr_frame, dtype=np.uint8)
        k, dequantized_Cr_frame = q_deq(Cr_frame, 1<<q_step)
        rate = bytes_per_grayframe(k)
        distortion = MSE(Cr_frame, dequantized_Cr_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_Cb_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        Cb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))[:,:,2]
        dequantized_Cb_frame = np.empty_like(Cb_frame)
        k = np.empty_like(Cb_frame, dtype=np.uint8)
        k, dequantized_Cb_frame = q_deq(Cb_frame, 1<<q_step)
        rate = bytes_per_grayframe(k)
        distortion = MSE(Cb_frame, dequantized_Cb_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

only_Y_points = only_Y_RD_curve(img)
only_Cb_points = only_Cb_RD_curve(img)
only_Cr_points = only_Cr_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YCrCb_points), c='m', marker="x",
           label='$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Cr}} = \Delta_{\mathrm{Cb}}$')
pylab.plot(*zip(*only_Y_points), c='r', marker="o",
           label='Only Y')              
pylab.plot(*zip(*only_Cr_points), c='g', marker="o",
           label='Only Cr')              
pylab.plot(*zip(*only_Cb_points), c='b', marker="o",
           label='Only Cb')              
pylab.title("RD Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

Conclusions:
1. The distortions are not additive.
2. The slopes of the curves for different quantization steps are different.

### RD using quantization in the YCoCg domain ($\Delta_{\text{Y}} = \Delta_{\text{Co}} = \Delta_{\text{Cg}}$)

In [ ]:
def YCoCg_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        k, dequantized_YCoCg_frame = q_deq(YCoCg_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame.astype(np.int16))
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

YCoCg_points = YCoCg_RD_curve(img)

### RD curves of each YCoCg channel

In [ ]:
def only_Y_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        Y_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))[:,:,0]
        dequantized_Y_frame = np.empty_like(Y_frame)
        #k = np.empty_like(Y_frame)
        k, dequantized_Y_frame = q_deq(Y_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_grayframe(k)
        distortion = MSE(Y_frame, dequantized_Y_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_Co_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        Co_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))[:,:,1]
        dequantized_Co_frame = np.empty_like(Co_frame)
        #k = np.empty_like(Co_frame)
        k, dequantized_Co_frame = q_deq(Co_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_grayframe(k)
        distortion = MSE(Co_frame, dequantized_Co_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

def only_Cg_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        Cg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))[:,:,2]
        dequantized_Cg_frame = np.empty_like(Cg_frame)
        #k = np.empty_like(Cg_frame)
        k, dequantized_Cg_frame = q_deq(Cg_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_grayframe(k)
        distortion = MSE(Cg_frame, dequantized_Cg_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

print(img.dtype)
only_Y_points = only_Y_RD_curve(img)
only_Co_points = only_Co_RD_curve(img)
only_Cg_points = only_Cg_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YCoCg_points), c='m', marker="x",
           label='$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Co}} = \Delta_{\mathrm{Cg}}$')
pylab.plot(*zip(*only_Y_points), c='r', marker="o",
           label='Only Y')              
pylab.plot(*zip(*only_Co_points), c='g', marker="o",
           label='Only Co')              
pylab.plot(*zip(*only_Cg_points), c='b', marker="o",
           label='Only Cg')              
pylab.title("RD Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

Conclusions:
1. The distortions are not additive.
2. The slopes of the curves for different quantization steps are different.
3. The channel Y should be quantized less.

### RD using channel gains in YCoCg domain

In [ ]:
def YCoCg_RD_curve(RGB_frame):
    RD_points = []
    #for q_step in range(0, 8):
    for q_step in range(1, 256):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        #k, dequantized_YCoCg_frame = q_deq(YCoCg_frame, 1<<q_step)
        k, dequantized_YCoCg_frame = q_deq(YCoCg_frame, q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

YCoCg_points = YCoCg_RD_curve(img)

relative_Y_gain = 3/2
relative_Co_gain = 1
relative_Cg_gain = 3/2
def YCoCg_RD_curve_with_gains(RGB_frame):
    RD_points = []
    #for q_step in range(0, 8):
    for q_step in range(1, 256):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        dequantized_YCoCg_frame = np.empty_like(YCoCg_frame)
        k = np.empty_like(YCoCg_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_frame[:,:,0] = q_deq(YCoCg_frame[:,:,0], (1<<q_step)/relative_Y_gain)
        k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], (1<<q_step)/relative_Co_gain)
        k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], (1<<q_step)/relative_Cg_gain)
        #k[0], dequantized_YCoCg_frame[0] = q_deq(YCoCg_frame[0], q_step/relative_Y_gain)
        #k[1], dequantized_YCoCg_frame[1] = q_deq(YCoCg_frame[1], q_step/relative_Co_gain)
        #k[2], dequantized_YCoCg_frame[2] = q_deq(YCoCg_frame[2], q_step/relative_Cg_gain)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

YCoCg_gains_points = YCoCg_RD_curve_with_gains(img)

In [ ]:
pylab.figure(dpi=150)
pylab.scatter(*zip(*YCoCg_points), c='g', marker=".", s=0.5,
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$')
pylab.scatter(*zip(*YCoCg_gains_points), c='m', marker=".", s=0.5,
           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(relative_Y_gain) + '\Delta_{\mathrm{Cg}}' +
           ';\Delta_{\mathrm{Cg}}=' + "{:3.1f}".format(relative_Cg_gain) + '\Delta_{\mathrm{Co}}$')
pylab.title("Performance of Quantization in Different Domains")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

### Comparing the three domains using the same quantization steps

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_points), c='b', marker="x",
           label='$\Delta_{\mathrm{R}}=\Delta_{\mathrm{G}}=\Delta_{\mathrm{B}}$')
pylab.plot(*zip(*YCrCb_points), c='r', marker="x",
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$')
#pylab.plot(*zip(*YYCrCb_gains_points), c='m', marker="x",
#           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(Y_gain) + '\Delta_{\mathrm{Cr}}' +
#           ';\Delta_{\mathrm{Cb}}=' + "{:3.1f}".format(Cb_gain) + '\Delta_{\mathrm{Cr}}$')
pylab.plot(*zip(*YCoCg_points), c='g', marker="x",
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$')
pylab.title("Performance of Quantization in Different Domains")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

Conclusions:
1. In general, quantization is more effective in the transformed domain.
3. At low bit-rates, it's better to quantize YCoCb than to quantize YCrCb. 

In [ ]:
Cr_gain = 1.0 # 2.4754
Cb_gain = 3.25832/2.4754
Y_gain = 3/2.4754
def YYCrCb_RD_curve_with_gains(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        YYCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YYCrCb_frame = np.empty_like(YYCrCb_frame)
        k = np.empty_like(YYCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YYCrCb_frame[:,:,0] = q_deq(YYCrCb_frame[:,:,0], (1<<q_step)/Y_gain)
        k[:,:,1], dequantized_YYCrCb_frame[:,:,1] = q_deq(YYCrCb_frame[:,:,1], (1<<q_step)/Cr_gain)
        k[:,:,2], dequantized_YYCrCb_frame[:,:,2] = q_deq(YYCrCb_frame[:,:,2], (1<<q_step)/Cb_gain)
        rate = bytes_per_frame(k)
        dequantized_YYCrCb_frame = dequantized_YYCrCb_frame.astype(np.uint8)
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YYCrCb_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
    return RD_points

YYCrCb_gains_points = YYCrCb_RD_curve_with_gains(img)

Conclusions:
1. In general, quantization is more effective in the transformed domain considering the RD plane.
2. $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ is slightly better than $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$, at low bit-rates, and viceversa.
3. At low bit-rates, tt's better to quantize YCoCb than to quantize YCrCb. 

## Is $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ the best quantization in YCrCb?
No, $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$ is better than $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ at high bit-rates.

## Is $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ optimal at low bit-rates quantizing YCrCb?

In [ ]:
N=5
def only_Y_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YCrCb_frame = np.empty_like(YCrCb_frame)
        k = np.empty_like(YCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCrCb_frame[:,:,0] = q_deq(YCrCb_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_YCrCb_frame[:,:,1] = q_deq(YCrCb_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCrCb_frame[:,:,2] = q_deq(YCrCb_frame[:,:,2], 1<<N)
        rate = bytes_per_frame(k)
        dequantized_YCrCb_frame = dequantized_YCrCb_frame.astype(np.uint8)
        assert dequantized_YCrCb_frame.all() >= 0
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YCrCb_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def only_Cb_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YCrCb_frame = np.empty_like(YCrCb_frame)
        k = np.empty_like(YCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCrCb_frame[:,:,0] = q_deq(YCrCb_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCrCb_frame[:,:,1] = q_deq(YCrCb_frame[:,:,1], 1<<q_step)
        k[:,:,2], dequantized_YCrCb_frame[:,:,2] = q_deq(YCrCb_frame[:,:,2], 1<<N)
        rate = bytes_per_frame(k)
        assert dequantized_YCrCb_frame.all() >= 0
        dequantized_YCrCb_frame = dequantized_YCrCb_frame.astype(np.uint8)
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YCrCb_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def only_Cr_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCrCb_frame = YCrCb.from_RGB(RGB_frame.astype(np.uint8))
        dequantized_YCrCb_frame = np.empty_like(YCrCb_frame)
        k = np.empty_like(YCrCb_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCrCb_frame[:,:,0] = q_deq(YCrCb_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCrCb_frame[:,:,1] = q_deq(YCrCb_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCrCb_frame[:,:,2] = q_deq(YCrCb_frame[:,:,2], 1<<q_step)
        rate = bytes_per_frame(k)
        dequantized_YCrCb_frame = dequantized_YCrCb_frame.astype(np.uint8)
        assert dequantized_YCrCb_frame.all() >= 0
        dequantized_RGB_frame = YCrCb.to_RGB(dequantized_YCrCb_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

only_Y_points = only_Y_RD_curve(img)
only_Cb_points = only_Cb_RD_curve(img)
only_Cr_points = only_Cr_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YYCrCb_gains_points), c='m', marker="x",
           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(Y_gain) + '\Delta_{\mathrm{Cr}}' +
           ';\Delta_{\mathrm{Cb}}=' + "{:3.1f}".format(Cb_gain) + '\Delta_{\mathrm{Cr}}$')
pylab.plot(*zip(*only_Y_points), c='r', marker="o",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Cb_points), c='g', marker="o",
           label='$\Delta_{\mathrm{Cb}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Cr_points), c='b', marker="o",
           label='$\Delta_{\mathrm{Cr}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<N))              
pylab.title("RD Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

No, there are combinations of $\Delta_{\mathrm{Y}}$, $\Delta_{\mathrm{Cr}}$, and $\Delta_{\mathrm{Cb}}$ better than $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}}; \Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$ at low bit-rates.

## Is $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$ optimal quantizing YCoCb?

In [ ]:
N=4
def only_Y_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        dequantized_YCoCg_frame = np.empty_like(YCoCg_frame)
        k = np.empty_like(YCoCg_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_frame[:,:,0] = q_deq(YCoCg_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<N)
        #k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<q_step)
        #k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<q_step)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def only_Co_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        dequantized_YCoCg_frame = np.empty_like(YCoCg_frame)
        k = np.empty_like(YCoCg_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_frame[:,:,0] = q_deq(YCoCg_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<q_step)
        k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<N)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def only_Cg_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        dequantized_YCoCg_frame = np.empty_like(YCoCg_frame)
        k = np.empty_like(YCoCg_frame, dtype=np.uint8)
        k[:,:,0], dequantized_YCoCg_frame[:,:,0] = q_deq(YCoCg_frame[:,:,0], 1<<N)
        k[:,:,1], dequantized_YCoCg_frame[:,:,1] = q_deq(YCoCg_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCoCg_frame[:,:,2] = q_deq(YCoCg_frame[:,:,2], 1<<q_step)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

only_Y_points = only_Y_RD_curve(img)
only_Co_points = only_Co_RD_curve(img)
only_Cg_points = only_Cg_RD_curve(img)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*only_Y_points), c='r', marker="o",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Co_points), c='m', marker="o",
           label='$\Delta_{\mathrm{Co}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cg}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*only_Cg_points), c='b', marker="o",
           label='$\Delta_{\mathrm{Cg}}~\mathrm{varies},~\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=$' + '{}'.format(1<<N))              
pylab.plot(*zip(*YCoCg_points), c='g', marker="x",
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$')
pylab.title("RD Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

At least, using the same experiment that before, $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Co}}=\Delta_{\mathrm{Cg}}$ seems to be near optimal quantizing YCoCb.

## Ignore the rest ...

## Some experiments showing the impact of the lack of orthogonality

In [ ]:
def _YCbCr_RD_curve(RGB_frame, N):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        dequantized_YCbCr_frame = np.empty_like(YCbCr_frame)
        k = np.empty_like(YCbCr_frame)
        k[:,:,0], dequantized_YCbCr_frame[:,:,0] = q_deq(YCbCr_frame[:,:,0], 1<<q_step)
        k[:,:,1], dequantized_YCbCr_frame[:,:,1] = q_deq(YCbCr_frame[:,:,1], 1<<N)
        k[:,:,2], dequantized_YCbCr_frame[:,:,2] = q_deq(YCbCr_frame[:,:,2], 1<<N)
        rate = bytes_per_frame(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

_YCbCr_points_8 = _YCbCr_RD_curve(frame, 8)
_YCbCr_points_7 = _YCbCr_RD_curve(frame, 7)
_YCbCr_points_6 = _YCbCr_RD_curve(frame, 6)
_YCbCr_points_5 = _YCbCr_RD_curve(frame, 5)
_YCbCr_points_4 = _YCbCr_RD_curve(frame, 4)
_YCbCr_points_3 = _YCbCr_RD_curve(frame, 3)

In [ ]:
1<<4

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YCbCr_points), c='r', marker="x",
           label='$\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$')
pylab.plot(*zip(*_YCbCr_points_8), c='b', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<8))
pylab.plot(*zip(*_YCbCr_points_7), c='g', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<7))
pylab.plot(*zip(*_YCbCr_points_6), c='c', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<6))
pylab.plot(*zip(*_YCbCr_points_5), c='m', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<5))
pylab.plot(*zip(*_YCbCr_points_4), c='y', marker="x",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<4))
pylab.plot(*zip(*_YCbCr_points_3), c='k', marker="o",
           label='$\Delta_{\mathrm{Y}}~\mathrm{varies},~\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}=$' + '{}'.format(1<<3))
pylab.plot(*zip(*YCbCr_gains_points), c='m', marker="+",
           label='$\Delta_{\mathrm{Y}}=' + "{:3.1f}".format(Y_gain) + '\Delta_{\mathrm{Cr}}' +
           ';\Delta_{\mathrm{Cb}}=' + "{:3.1f}".format(Cb_gain) + '\Delta_{\mathrm{Cr}}$')
pylab.title("The lack of non-orthogonality in the YCrCb transform")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

From this experiment we conclude that:
1. The luma should not be "deleted" from the code-stream (see curve $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$).
2. There are better combinations than $\Delta_{\mathrm{Y}}=\Delta_{\mathrm{Cr}}=\Delta_{\mathrm{Cb}}$ and $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}};\Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$.

### YCrCb

It's possible to find better combinations than $\Delta_{\mathrm{Y}}=1.2\Delta_{\mathrm{Cr}};\Delta_{\mathrm{Cb}}=1.3\Delta_{\mathrm{Cr}}$.

Notice that, at least visually, it does not make sense to use $\Delta_{\mathrm{Y}}\ge 8$ because the reconstructed image will be very dark or even black. The same holds for YCoCg.

In [ ]:
ycc = RGB_to_YCbCr(frame.astype(np.uint8))
ycc[:,:,0] = 0
frame2 = YCbCr_to_RGB(ycc)
print(frame2.min(), frame2.max())
show_frame(frame2, "$\Delta_{\mathrm{Y}} \ge 8$" + " (YCbCr domain)")

### YCoCg

Notice that, at least visually, it does not make sense to use $\Delta_{\mathrm{Y}}\ge 8$ because the reconstructed image will be very dark or even black. The same holds for YCoCg.

In [ ]:
ycc = RGB_to_YCoCg(frame)
ycc[:,:,0]= 0
frame2 = YCoCg_to_RGB(ycc)
print(frame2.min(), frame2.max())
show_frame(frame2, "$\Delta_{\mathrm{Y}} \ge 8$" + " (YCoCg domain)")

In [ ]:
def YCbCr_RD_curve_only_Y(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        YCbCr_frame[:,:,1] = 0
        YCbCr_frame[:,:,2] = 0
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        k[:,:,0] = 0
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def YCbCr_RD_curve_only_Cb(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        YCbCr_frame[:,:,0] = 0
        YCbCr_frame[:,:,2] = 0
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

def YCbCr_RD_curve_only_Cr(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint8))
        YCbCr_frame[:,:,0] = 0
        YCbCr_frame[:,:,1] = 0
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint8)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

only_Y_curve = YCbCr_RD_curve_only_Y(frame)
only_Cb_curve = YCbCr_RD_curve_only_Cb(frame)
only_Cr_curve = YCbCr_RD_curve_only_Cr(frame)

In [ ]:
pylab.figure(dpi=150)
pylab.scatter(*zip(*only_Y_curve), s=2, c='r', marker="o", label='only Y')
pylab.plot(*zip(*only_Y_curve), c='r', marker="o")
pylab.scatter(*zip(*only_Cb_curve), s=2, c='g', marker="o", label='only Cb')
pylab.plot(*zip(*only_Cb_curve), c='g', marker="o")
pylab.scatter(*zip(*only_Cr_curve), s=2, c='b', marker="o", label='only Cr')
pylab.plot(*zip(*only_Cr_curve), c='b', marker="o")
pylab.title("R/D Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
def YCbCr_RD_curve_2(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        print(q_step, end=' ')
        YCbCr_frame = RGB_to_YCbCr(RGB_frame.astype(np.uint16))
        YCbCr_frame[:,:,1] //= 2
        YCbCr_frame[:,:,2] //= 2
        k, dequantized_YCbCr_frame = q_deq(YCbCr_frame, 1<<q_step)
        dequantized_YCbCr_frame[:,:,1] *= 2
        dequantized_YCbCr_frame[:,:,2] *= 2
        rate = byte_rate(k)
        dequantized_YCbCr_frame = dequantized_YCbCr_frame.astype(np.uint16)
        dequantized_RGB_frame = YCbCr_to_RGB(dequantized_YCbCr_frame)
        distortion = RMSE(RGB_frame, dequantized_RGB_frame)
        RD_points.append((rate, distortion))
    return RD_points

YCbCr_quantization_2 = YCbCr_RD_curve_2(frame)

In [ ]:
pylab.figure(dpi=150)
pylab.scatter(*zip(*RGB_quantization), s=2, c='b', marker="o", label='RGB quantization')
pylab.plot(*zip(*RGB_quantization), c='b', marker="o")
pylab.scatter(*zip(*YCbCr_quantization), s=2, c='r', marker="o", label='YCbCr quantization')
pylab.plot(*zip(*YCbCr_quantization), c='r', marker="o")
pylab.scatter(*zip(*YCbCr_quantization_2), s=2, c='g', marker="o", label='YCbCr quantization 2')
pylab.plot(*zip(*YCbCr_quantization_2), c='g', marker="o")
pylab.title("R/D Only Quantization")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
YCbCr_test_frame = np.array([255, 0, 0], dtype=np.int16).reshape((1,1,1))
print(YCbCr_to_RGB(YCbCr_test_frame))

In [ ]:
np.array([255, 0, 0], dtype=np.int16)

In [ ]:
YCbCr_test_frame = np.zeros_like(frame).astype(np.uint16)

In [ ]:
type(YCbCr_test_frame[0,0,0])

In [ ]:
YCbCr_test_frame[1,1,2] = 255

In [ ]:
RGB_test_frame = YCbCr_to_RGB(YCbCr_test_frame)

In [ ]:
print(average_energy(RGB_test_frame))

In [ ]:
show_frame(RGB_test_frame, "")